# [Wiener's Attack](https://en.wikipedia.org/wiki/Wiener%27s_attack)

#### Group 7: Austin Beard, Junior Saintcius, John Owens

## Preliminaries (Definitions and imports)

In [163]:
#Imports
import math

# Group 7's (N, e) pair
N_big = 947915625495728052916879147697360002848381185192346505887337318484424613804479379229500825154943023456531602566274207306369591959850608246425659384399592991634207719975226030904151882757589795301726622205405989011351842286767545223253169802897983637243167859583565776182244623125858428782210670308167292094521748561446916209557796073456184675807739356933731724960127222046486932110371797638731014250317825447454900437565086817233575744240640382684548268127956137173595899977563370261355506081974467333993545072663993466761303977372442140485495352997067642695566437175979143720808011678287958025730556920566713423095016601310050215390374360676665989625828596980532017231388237244123248528625713956773078115159965411732942413787894237594334962280338475577955633910836559629956564026617798854751095370708381339497715897167336403274224664697600147738325680779222350682292639751395534249345846969763993219058901236669322261406103455020265135135221755871135240191771715762695473152226592913073165991509715785601507547540424028508272006742607644751844712565734384457757831906046716752425927716813761510281226714973672364905890290774159582494385327020386478310127596750245584058550668842273035173930485514220124363571210008854397877759878109
e_big = 704547874817628536678711224260347170080780157225302414633058087631273478699354942316191545183552970491256712479930884728272203750707194999236141913284914475190760565306955811426771755057311324861065120995469067811477070684340373917672377871037102399678405113518962175705845425490703244686038865941492819081360216141451957694835143605736374176464955479837962508015825872034637667723449908810156455398586426946187983417091803239944119556424937448702560147449296000358404512385942224137056969472773149280435494014441168305623041946847983160401839626890022631063357714302185156647516439105432586919947268965890950159670021155797827170793898880619668912414078180761247776607171716089112584888939756383508383552798449319245738264857608637364581842457448498549984823842222174304406071636079197079103741311287709390835892500282167064692991655929368336712543591994988841130885756415329962100690973384132459114994258583928548266879551395010165988053756662119802587150754631529097435381252816428859268931378932500218048710530873387464697995565066027920879230108859236321171012617250021403409639363576159429016313582199500594524217662031829176343714208316657171662175877320736019254117219869310765030193442816632528910296054592307278119395320563


### Testing definitions

In [197]:
def gen_pqN(bits):
    """
    Input: number of bits for primes
    Output: p, q, N=p*q
    Example: gen_pqN(128) returns 128-bit p, 128-bit q, 256-bit N
    """
    primes = []
    
    while(len(primes)< 2):
        candidate = ZZ.random_element(2**bits-1)
        if candidate.is_prime():
            if (len(primes) == 1 
            and candidate > primes[0]
            and candidate < primes[0]*2):
                primes.append(candidate)
            elif len(primes) == 0:
                primes.append(candidate)
    q, p = primes[0], primes[1]
    
    return p, q, p*q

def gen_e(p,q):
    phiN = (p-1) * (q-1)
    e = ZZ.random_element(phiN)
    while gcd(phiN, e) != 1:
        e = ZZ.random_element(phiN)
    return e

def gen_d(e, phiN):
    return inverse_mod(e, phiN)

def gen_de_smalld(p, q):
    phiN = (p-1) * (q-1)
    N = p*q
    start = math.ceil(((N^(0.25))/3))-1000
    d = gen_d_smalld(p, q, phiN, start)
    e = gen_e_smalld(d, phiN)
    while e > N^(3/2):
        start += 1
        d = gen_d_smalld(p, q, phiN, start)
        e = gen_e_smalld(d, phiN)
    return d, e

def gen_d_smalld(p, q, phiN, start=1):
    d = start
    while gcd(phiN, d) != 1:
        d += 1    
    return d

def gen_e_smalld(d, phiN):
    return gen_d(d, phiN)
    

### Essential definitions

In [194]:
def getConvergents(N,e):
    return (e/N).continued_fraction().convergents()

# def calcPhiN(convergents, e):
#     for conv in convergents:
#         if conv != 0:
#             phiN = conv^(-1) * e - 1
#             if phiN.denom() == 1:
#                 return phiN
#     raise Exception('No factorizations found!')
    
def calcPhiN(convergents, e):
    for conv in convergents:
        if conv != 0:
#             phiN = (conv^(-1) * e) - 1
            num = (conv.denominator() * e) - 1
            denom  = conv.numerator()
            phiN = num/denom
            if phiN.denom() == 1:
                return phiN
    raise Exception('No factorizations found!')

def findRoots(phiN, N):
    return solve(x^2 - ((N - phiN)+1)*x + N, x, solution_dict=True)

def Wiener(N, e):
    """
    Input: integers N and e
    Output: integers p, q, and d
    """
    
    #Do the attack here
    convergents = getConvergents(N, e)
    phiN = calcPhiN(convergents, e)
    roots = findRoots(phiN, N)
    p = roots[0][x]
    q = roots[1][x]
    d = gen_d(e, phiN)
    
    #return results
    return p, q, d

## Tests

### Following the example on Wikipedia:

In [166]:
#Test continued fraction (example from Wikipedia) - works
N = 90581
e = 17993

print(continued_fraction(17993/90581))
print((17993/90581).continued_fraction().convergents())

[0; 5, 29, 4, 1, 3, 2, 4, 3]
[0, 1/5, 29/146, 117/589, 146/735, 555/2794, 1256/6323, 5579/28086, 17993/90581]


In [195]:
N_wiki = 90581
e_wiki = 17993
p_wiki = 379
q_wiki = 239
phiN_wiki = (p_wiki-1)*(q_wiki-1)

print('Original p,q: {},{}'.format(p_wiki,q_wiki))
print('Original d: {}'.format(gen_d(e_wiki, phiN_wiki)))
p_recov, q_recov, d_recov = Wiener(N_wiki, e_wiki)
print('Recovered p,q: {},{}'.format(p_recov, q_recov))
print('Recovered d: {}'.format(d_recov))

Original p,q: 379,239
Original d: 5
Recovered p,q: 379,239
Recovered d: 5


### Test generating small 'd' first

In [168]:
d,e = gen_de_smalld(p_wiki, q_wiki)
print((d,e))

mt = 5
ct = pow(mt,d,N_wiki)
mct = pow(ct,e,N_wiki)

print(mt, ct, mct)

(-991, 12437)
5 6007 5


### Generalize:
## This isn't working..for now. d is too large.

In [198]:
def testWienerRandom(bits):
    """
    Finding d, where p and q are "bits" bits each.
    """
    p, q, N = gen_pqN(bits)
#     phiN = (p-1)*(q-1)
    d,e = gen_de_smalld(p,q)

    while d >= (N^(1/4))/3:
        p, q, N = gen_pqN(bits)
#         phiN = (p-1)*(q-1)
        d,e = gen_de_smalld(p,q)
    
    print('Original p,q: {},{}'.format(p,q))
    print('Original d,e: {},{}'.format(d,e))
    p_recov, q_recov, d_recov = Wiener(N, e)
    print('Recovered p,q: {},{}'.format(p_recov, q_recov))
    print('Recovered d: {}'.format(d_recov))
    
testWienerRandom(128)

Original p,q: 318682516130230716491204857243837331899,309035911827618983107233014928380091913
Original d,e: 5905008704476424219,19784553682639035808213872809893023796162101173033623223317001881392802478867
Recovered p,q: -1/2*sqrt(374272638616317663840684430841220407375220736329374097730455800377754061832373354582192769483914509399177352519074050798940021862796940489400915882485893) + 19346127225269600811942653453821176072154354928246366073125852101643096917321/2,1/2*sqrt(374272638616317663840684430841220407375220736329374097730455800377754061832373354582192769483914509399177352519074050798940021862796940489400915882485893) + 19346127225269600811942653453821176072154354928246366073125852101643096917321/2
Recovered d: 4


In [170]:
#Testing on multiple (N, e) pairs
# N = 256 bits

p, q, N = gen_pqN(128)
print(p,q,N)

# p, q, d = Wiener(N, e)

207582472920375578813462022358470584381 98520343073934046407434406230274777211 20451096448251025899790855439626634464347766195169056067951927283031751341391


In [171]:
print(Wiener(N, e))

(-5*I*sqrt(818043857930041035991634217585065378573910647806762242718077091321269828790) + 2371, 5*I*sqrt(818043857930041035991634217585065378573910647806762242718077091321269828790) + 2371, 1644375367713357393245224365974642957654399468936966798098570980383673823)


In [172]:
Primes().unrank(5)

13

In [ ]:
# sage: p = 320273329987802959821281693988321547291
# sage: q = 149649074209532604
# sage: q = 149649074209532604317640663289310668957
# sage: N = p*q
# sage: phiN = (p-1)*(q-1)
# sage: solve(x^2 - ((N - phiN)+1)*x + N, x, solution_dict=True)
# [{x: 320273329987802959821281693988321547291},
#  {x: 149649074209532604317640663289310668957}]
# sage: def calcPhiN(convergents, e):
# ....:     phiNs = []
# ....:     for conv in convergents:
# ....:         if conv != 0:
# ....:             phiN = conv^(-1) * e - 1
# ....:             if phiN.denom() == 1:
# ....:                 phiNs.append(phiN)
# ....:     #                 return phiN
# ....:     if len(phiNs) == 0:
# ....:         raise Exception('No factorizations found!')
# ....:
# ....:     return phiNs
# ....:
# sage: e = 23996788267625896323473767121972395254189012995588274077311938120900663209053
# sage: d = 4932049738791738397
# sage: phiNs = calcPhiN((e/N).continued_fraction().convergents(), e)
# sage: phiNs
# [23996788267625896323473767121972395254189012995588274077311938120900663209052,
#  47993576535251792646947534243944790508378025991176548154623876241801326418105,
#  47928607326678849157541486379966219744101079672005546355598566125376521145486]
# sage: phiN
# 47928607326678849157541486379966219743631157267808210791459643768098888929240
# sage: phiN - 47928607326678849157541486379966219744101079672005546355598566125376521145486
# -469922404197335564138922357277632216246